In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('jamaMortalityMeta.xlsx')
data = data.set_index(data['index'], drop=True)

In [3]:
data

,index,italylDeaths,italyCaes,italyCaseFatality,chinaDeaths,chinaCases,chinaCaseFatality
index,,,,,,,
0-9,0-9,0,NaN,0.000,0,NaN,0.0000
10-19,10-19,0,NaN,0.000,1,500.000000,0.0020
20-29,20-29,0,NaN,0.000,7,3500.000000,0.0020
30-39,30-39,4,1333.333333,0.003,18,9000.000000,0.0020
40-49,40-49,10,2500.000000,0.004,38,9500.000000,0.0040
50-59,50-59,43,4300.000000,0.010,130,12621.359223,0.0103
60-69,60-69,139,3971.428571,0.035,309,8583.333333,0.0360
70-79,70-79,578,4515.625000,0.128,312,3900.000000,0.0800
80-,80-,850,4207.920792,0.202,208,1405.405405,0.1480


In [4]:
import statsmodels.stats.proportion as prop
import math

metaAnalyses = {}

for i in range(0, 9):
    italyCF = data.iloc[i, 3]
    chinaCF = data.iloc[i, 6]
    italyDeaths = data.iloc[i, 1]
    chinaDeaths = data.iloc[i, 4]
    if (math.isnan(data.iloc[i, 2]) or math.isnan(data.iloc[i, 3]) ):
        metaAnalyses[data.index[i]] = {'caseFatality' : np.nan, 'se' : np.nan, 'lci' :np.nan, 'uci':np.nan}
        continue
 
    italyCases = int(data.iloc[i, 2])
    chinaCases = int(data.iloc[i, 5])
    
    italySE = italyCF - prop.proportion_confint(italyDeaths, italyCases)[0]
    chinaSE = chinaCF - prop.proportion_confint(chinaDeaths, chinaCases)[0]
    print(chinaCases)
    
    italyWeightedProportion = italyCF * italyCases/(italyCases+chinaCases)
    italyWeightedProportionSE = italySE* italyCases/(italyCases+chinaCases)
    
    chinaWeightedProportion = chinaCF * italyCases/(italyCases+chinaCases)
    chinaWeightedProportionSE = chinaSE * italyCases/(italyCases+chinaCases)
    
    totalWeightedProportion = italyWeightedProportion+chinaWeightedProportion
    totalWeightedProportionSE = italyWeightedProportionSE+chinaWeightedProportionSE
    metaAnalyses[data.index[i]] = {'caseFatality' : totalWeightedProportion, 'se' : totalWeightedProportionSE, 
                           'lci' :totalWeightedProportion-1.96*totalWeightedProportionSE,
                           'uci':totalWeightedProportion+1.96*totalWeightedProportionSE }
    



9000
9500
12621
8583
3900
1405


In [5]:
df = pd.DataFrame(metaAnalyses)
df = df.T
df['ageCat'] = df.index
df['lowerAgeBound'] = np.nan
df['upperAgeBound'] = np.nan

In [6]:
for row in df.iterrows():
    df.set_value(row[0], 'lowerAgeBound', row[1]['ageCat'].split('-')[0])
    print(row[1]['ageCat'].split('-')[1])
    if (len(row[1]['ageCat'].split('-')) > 1):
        df.set_value(row[0], 'upperAgeBound', row[1]['ageCat'].split('-')[1])
df.set[8,]

9
19
29
39
49
59
69
79



/Users/burke/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
/Users/burke/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


ValueError: could not convert string to float: 

In [7]:
df.loc['80-', 'upperAgeBound'] = 120
df.to('mortalityByAge.csv')

In [8]:
# sketch...out the code for convering age into a categorical and getting the baseline values

In [9]:
df

,caseFatality,se,lci,uci,ageCat,lowerAgeBound,upperAgeBound
0-9,NaN,NaN,NaN,NaN,0-9,0.0,9.0
10-19,NaN,NaN,NaN,NaN,10-19,10.0,19.0
20-29,NaN,NaN,NaN,NaN,20-29,20.0,29.0
30-39,0.000645,0.000498,-0.000331,0.001621,30-39,30.0,39.0
40-49,0.001667,0.000780,0.000138,0.003195,40-49,40.0,49.0
50-59,0.005159,0.001203,0.002800,0.007517,50-59,50.0,59.0
60-69,0.022458,0.003053,0.016474,0.028442,60-69,60.0,69.0
70-79,0.111601,0.009788,0.092417,0.130785,70-79,70.0,79.0
80-,0.262375,0.022951,0.217391,0.307360,80-,80.0,120.0


In [10]:
age=32
dDimer= .6
ferritin=100
il6 = 1

In [11]:
from uncertainties import ufloat

cat = pd.cut(x=[age], bins=[0, 9, 19, 29, 39, 49, 59, 69, 79, 120])
ageMean = df.iloc[cat.codes[0], 0]
ageSE = df.iloc[cat.codes[0], 1]
ageMortality = ufloat(ageMean, ageSE)

In [12]:
ddimerCat = pd.cut(x=[dDimer], bins=[0, 0.5, 1, 1000])
ddimerCoeffs = [ufloat(1, 0), ufloat(1.96,0.73), ufloat(20.04,6.89) ]
ddimerOR = ddimerCoeffs[ddimerCat.codes[0]]

In [13]:
ferritinCat = pd.cut(x=[ferritin], bins=[0, 300, 10000])
ferritinCoeffs = [ufloat(1, 1), ufloat(9.1, 3.6) ]
ferritinOR = ferritinCoeffs[ferritinCat.codes[0]]

In [14]:
il6OR = il6 * ufloat(1.12, 0.046)

In [15]:
totalBiomarkerOR = ddimerOR * ferritinOR * il6OR

In [16]:
totalBiomarkerOR

2.1952000000000003+/-2.3442486270871528

In [17]:
combinedOdds = ageMortality/(1-ageMortality)*totalBiomarkerOR

In [18]:
combinedProb = combinedOdds/(combinedOdds+1)

In [29]:
combinedProb.std_dev

0.001861920164909948

In [20]:
mortalityDF = pd.read_csv('mortalityByAge.csv')


In [27]:
mortalityDF.index=mortalityDF.ageCat
mortalityDF = mortalityDF.drop(labels=mortalityDF.columns[0], axis='columns')

In [28]:
mortalityDF

,caseFatality,se,lci,uci,ageCat,lowerAgeBound,upperAgeBound
ageCat,,,,,,,
0-9,NaN,NaN,NaN,NaN,0-9,0.0,9.0
10-19,NaN,NaN,NaN,NaN,10-19,10.0,19.0
20-29,NaN,NaN,NaN,NaN,20-29,20.0,29.0
30-39,0.000645,0.000498,-0.000331,0.001621,30-39,30.0,39.0
40-49,0.001667,0.000780,0.000138,0.003195,40-49,40.0,49.0
50-59,0.005159,0.001203,0.002800,0.007517,50-59,50.0,59.0
60-69,0.022458,0.003053,0.016474,0.028442,60-69,60.0,69.0
70-79,0.111601,0.009788,0.092417,0.130785,70-79,70.0,79.0
80-,0.262375,0.022951,0.217391,0.307360,80-,80.0,120.0
